In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import requests
from datetime import datetime as dt

In [2]:
ALPHAVANTAGE_KEY = os.getenv('ALPHAVANTAGE_KEY')
assert ALPHAVANTAGE_KEY is not None, "You must export ALPHAVANTAGE_KEY..."
ALPHAVANTAGE_KEY

'032HZXCB7T21N7SC'

In [3]:
SAMPLE_START_DATE = dt.fromisoformat('2019-12-19').date()
SAMPLE_END_DATE = dt.fromisoformat('2024-04-30').date()
(SAMPLE_START_DATE, SAMPLE_END_DATE)

(datetime.date(2019, 12, 19), datetime.date(2024, 4, 30))

In [4]:
DIGITAL_CURRENCY_URL = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey={}&datatype=csv'.format(ALPHAVANTAGE_KEY)
SPY_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={apikey}&datatype=csv'.format(symbol='SPY', apikey=ALPHAVANTAGE_KEY)
QQQ_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={apikey}&datatype=csv'.format(symbol='QQQ', apikey=ALPHAVANTAGE_KEY)
(DIGITAL_CURRENCY_URL, SPY_URL, QQQ_URL)

('https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey=032HZXCB7T21N7SC&datatype=csv',
 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=SPY&outputsize=full&apikey=032HZXCB7T21N7SC&datatype=csv',
 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=QQQ&outputsize=full&apikey=032HZXCB7T21N7SC&datatype=csv')

In [5]:
%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format
sns.set_style("whitegrid")

mpl.rcParams.update({
                     'text.color' : "white",
                     'lines.linewidth': 3,
                     'font.size': 16.0,
                     'legend.facecolor': "182742",
                     'legend.edgecolor': 'CFB023',
                     'legend.fancybox': True,
                     'legend.framealpha': 1.,
                     'legend.loc': 'right',
                     'xtick.color': 'white',
                     'ytick.color': 'white',
                     'ytick.minor.visible': True,
                     'axes.facecolor': '#182742',
                     'axes.edgecolor': 'white',
#                      'axes.grid.axis': 'y',
#                      'axes.grid.which': 'minor',
                     'axes.labelcolor' : "white",
                     'axes.labelsize': "24",
                     'axes.labelweight': "bold",
                     'axes.titlesize': 28,
                     'axes.titleweight': 'bold',
                     'axes.titlepad': 20,
                     'figure.facecolor': '#182742',
#                      'xtick.bottom': False,
                    }
                   )

In [6]:
def total_return(prices):
  return prices.iloc[-1] / prices.iloc[0] - 1

# Download Bitcoin Historical Data

In [7]:
# TODO: Figure out how to add error checking
# r = requests.get(URL)
# data = r.json()
# assert 'Error Message' not in data.keys(), "Error Received. {}".format(data['Error Message'])

df = pd.read_csv(DIGITAL_CURRENCY_URL)
df['timestamp'] = pd.to_datetime(df.timestamp)
df.index = df.timestamp.dt.date
df = df[::-1]
print(df.columns)
df.rename(columns={"close": "btc_close"}, inplace=True)
# df = df.drop(['open (USD)', 'high (USD)', 'low (USD)', 'open (USD).1', 'high (USD).1', 'low (USD).1', 'close (USD).1'], axis=1)
df['pct_change'] = df['btc_close'].pct_change()
df[['timestamp', 'btc_close', 'pct_change']].to_csv('bitcoin_daily_returns.csv')
df[['timestamp', 'btc_close', 'pct_change']]
df

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume'], dtype='object')


,timestamp,open,high,low,btc_close,volume,pct_change
timestamp,,,,,,,
2023-07-23,2023-07-23,"29,793.6200","30,350.7000","29,733.5500","30,081.6100","4,513.8321",NaN
2023-07-24,2023-07-24,"30,081.6100","30,099.9900","28,850.0000","29,176.9800","14,484.2754",-0.0301
2023-07-25,2023-07-25,"29,176.9800","29,376.5100","29,046.0000","29,225.1400","7,139.2107",0.0017
2023-07-26,2023-07-26,"29,225.1100","29,686.4100","29,089.5400","29,353.2300","9,635.0350",0.0044
2023-07-27,2023-07-27,"29,350.8800","29,571.4200","29,075.0000","29,214.9200","7,327.2908",-0.0047
...,...,...,...,...,...,...,...
2024-05-13,2024-05-13,"61,453.0100","63,456.1800","60,738.5100","62,932.3700","9,170.7541",0.0241
2024-05-14,2024-05-14,"62,927.4400","63,103.0200","61,024.5100","61,539.8400","13,456.6131",-0.0221
2024-05-15,2024-05-15,"61,539.7000","66,480.0000","61,299.5700","66,244.6900","21,821.3886",0.0765


In [8]:
btc_monthly_returns = df.groupby([df.timestamp.dt.year, df.timestamp.dt.month])['btc_close'].apply(total_return)
btc_monthly_returns.to_csv("bitcoin_monthly_returns.csv")
btc_monthly_returns

timestamp  timestamp
2023       7           -0.0283
           8           -0.1268
           9            0.0452
           10           0.2379
           11           0.0647
           12           0.0926
2024       1           -0.0378
           2            0.4202
           3            0.1418
           4           -0.1300
           5            0.1217
Name: btc_close, dtype: float64

In [9]:

def read_equities(url, start_date, end_date, outfile):
    df = pd.read_csv(url)
    df['timestamp'] = pd.to_datetime(df.timestamp)
    df.index = df.timestamp.dt.date
    df = df[::-1]
    sample = df[start_date:end_date]
    # df.rename(columns={"close (USD)": "btc_close"}, inplace=True)
    # df = df.drop(['open (USD)', 'high (USD)', 'low (USD)', 'open (USD).1', 'high (USD).1', 'low (USD).1', 'close (USD).1'], axis=1)
    sample['pct_change'] = sample['close'].pct_change().copy()
    sample['growth_of_10k'] = (sample['pct_change'].iloc[1:] + 1.).cumprod() * 10000.
    sample['growth_of_10k'].iloc[0] = 10000.
#     sample.insert(4, 'growth_of_10k', 0.)
#     sample['growth_of_10k'].iloc[0] = 10000.0
# #     print(sample.iloc[1:]['growth_of_10k'])
#     print(sample['growth_of_10k'].shift(1))
#     print((1 + sample['pct_change']))
#     sample['growth_of_10k'] = sample['growth_of_10k'].shift(1) * (1 + sample['pct_change'])
    sample[['timestamp', 'close', 'pct_change', 'growth_of_10k']].to_csv(outfile)
    return sample[['timestamp', 'close', 'pct_change', 'growth_of_10k']]

In [10]:
spy_history = read_equities(SPY_URL, SAMPLE_START_DATE, SAMPLE_END_DATE, 'spy_history.csv')
spy_history

/var/folders/ls/5hpcb8qd44980kmt18m6krr40000gn/T/ipykernel_95139/43137134.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['pct_change'] = sample['close'].pct_change().copy()
/var/folders/ls/5hpcb8qd44980kmt18m6krr40000gn/T/ipykernel_95139/43137134.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['growth_of_10k'] = (sample['pct_change'].iloc[1:] + 1.).cumprod() * 10000.
/var/folders/ls/5hpcb8qd44980kmt18m6krr40000gn/T/ipykernel_95139/43137134.py:11: SettingWithCopyWarning: 
A value is t

,timestamp,close,pct_change,growth_of_10k
timestamp,,,,
2019-12-19,2019-12-19,320.9000,NaN,"10,000.0000"
2019-12-20,2019-12-20,320.7300,-0.0005,"9,994.7024"
2019-12-23,2019-12-23,321.2200,0.0015,"10,009.9720"
2019-12-24,2019-12-24,321.2300,0.0000,"10,010.2836"
2019-12-26,2019-12-26,322.9400,0.0053,"10,063.5712"
...,...,...,...,...
2024-04-24,2024-04-24,505.4100,-0.0005,"15,749.7663"
2024-04-25,2024-04-25,503.4900,-0.0038,"15,689.9346"
2024-04-26,2024-04-26,508.2600,0.0095,"15,838.5790"


In [11]:
qqq_history = read_equities(QQQ_URL, SAMPLE_START_DATE, SAMPLE_END_DATE, 'qqq_history.csv')
qqq_history

/var/folders/ls/5hpcb8qd44980kmt18m6krr40000gn/T/ipykernel_95139/43137134.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['pct_change'] = sample['close'].pct_change().copy()
/var/folders/ls/5hpcb8qd44980kmt18m6krr40000gn/T/ipykernel_95139/43137134.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['growth_of_10k'] = (sample['pct_change'].iloc[1:] + 1.).cumprod() * 10000.
/var/folders/ls/5hpcb8qd44980kmt18m6krr40000gn/T/ipykernel_95139/43137134.py:11: SettingWithCopyWarning: 
A value is t

,timestamp,close,pct_change,growth_of_10k
timestamp,,,,
2019-12-19,2019-12-19,210.8600,NaN,"10,000.0000"
2019-12-20,2019-12-20,211.7100,0.0040,"10,040.3111"
2019-12-23,2019-12-23,211.8100,0.0005,"10,045.0536"
2019-12-24,2019-12-24,211.9200,0.0005,"10,050.2703"
2019-12-26,2019-12-26,213.7900,0.0088,"10,138.9548"
...,...,...,...,...
2024-04-24,2024-04-24,426.5100,0.0034,"20,227.1649"
2024-04-25,2024-04-25,424.4500,-0.0048,"20,129.4698"
2024-04-26,2024-04-26,431.0000,0.0154,"20,440.1024"
